# Riiid! Answer Correctness Prediction - Intro
Team: EE258_F20_StudentPredictor

Author: Rendale Mark Taas

Last edited: 31-Dec-2020

This model was designed to use a complex network with an LSTM to extract time dependant trends. Since data is formatted as a time series, inputs are build on a per user basis. When training, a number of samples is pulled from each user. When testing, a database is saved and updated at each group/batch and takes into account new users. This database is in the variable 'test_user_dict'.

The hyperparameters are: the number of time steps in a series (variable timesteps), the number of samples per user (variable sampleperuser), the number of neurons per layer (variable lstm_neurons, ct_neurons, and dense_neurons), the learning rate (variable eta), and the number of training epochs (variable totalepoch).

In [ ]:
# Script for competition "Riiid! Answer Correctness Prediction"
# Competition summary can be found here:
# https://www.kaggle.com/c/riiid-test-answer-prediction/overview

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pandas as pd
import numpy as np
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import riiideducation

# Training Set Processing

In [ ]:
#Load training data from csv file
#Data types are explicitly specified to minimize memory usage
subset_size = int(6E6)
train_df = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=subset_size,
    dtype={
        'row_id': 'int64',
        'timestamp': 'int64',
        'user_id': 'int32',
        'content_id': 'int16',
        'content_type_id': 'int8',
        'task_container_id': 'int16',
        'user_answer': 'int8',
        'answered_correctly': 'int8',
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
    }
)
print(train_df.dtypes)
print('From .csv loading train_df: rows, columns =', train_df.shape)

In [ ]:
#Quick glimpse at the training data
train_df.iloc[80:100]

In [ ]:
#Remove unused columns
train_df.drop(['user_answer'], axis=1, inplace=True)
print('After dropping train_df: rows, columns =', train_df.shape)

In [ ]:
#Define data frame preprocesser (removes nans and converts bools)
mean_pqet = train_df['prior_question_elapsed_time'].mean()
def df_preprocess(in_df):
    in_df['prior_question_elapsed_time'].fillna(value=mean_pqet, inplace=True)
    in_df['prior_question_had_explanation'] = in_df[
        'prior_question_had_explanation'
    ].fillna(value=False).astype(bool).map({True:1, False:0})
    in_df.fillna(value=0, inplace=True)
    
df_preprocess(train_df)
print('After preprocessing train_df: rows, columns', train_df.shape)

In [ ]:
#Group by user and how many rows they have in the dataset
train_by_user_id = train_df.groupby(by='user_id')
user_list = list(train_by_user_id.groups.keys())
print('There are', len(user_list), 'users')
print('Average rows per user:', train_by_user_id.count().mean()[0])
print('Min rows of one user:', train_by_user_id.count().min()[0])
print('Max rows of one user:', train_by_user_id.count().max()[0])

#Delete all large variables not used past this point to conserve memory
del train_by_user_id

In [ ]:
#Group by question, and create dictionary
#Dictionary has the question's mean, standard deviation, and skew
train_by_content_id = train_df.loc[train_df.content_type_id==0].groupby(by='content_id')
content_id_dict = train_by_content_id.agg({'answered_correctly': [np.mean, np.std, 'skew']}).copy()
content_id_dict.columns = ['content_mean', 'content_std', 'content_skew']
content_id_dict.fillna(value=0, inplace=True)

#Delete all large variables not used past this point to conserve memory
del train_by_content_id

In [ ]:
#Group by container id, and create dictionary
#Dictionary has the container's mean, standard deviation, and skew
train_by_container = train_df.loc[train_df.content_type_id==0].groupby(by='task_container_id')
container_dict = train_by_container.agg({'answered_correctly': [np.mean, np.std, 'skew']}).copy()
container_dict.columns = ['container_mean', 'container_std', 'container_skew']
container_dict.fillna(value=0, inplace=True)

#Delete all large variables not used past this point to conserve memory
del train_by_container

In [ ]:
#From the questions metadata, create dictionary of part and tags
print('Start building questions_df at', time.ctime())
time_start = time.time()

questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions_df.drop(['bundle_id', 'correct_answer'], axis=1, inplace=True)
#Tagless questions are given the tag '-1'
questions_df.fillna(value=-1, inplace=True)
questions_df.rename(columns={'question_id':'content_id'}, inplace=True)
total_q = len(questions_df)

tag_list = []
first_tag_list = []
tag_counts = {}
for thisindex, thisstr in enumerate(questions_df.tags):
    temp_list = str(thisstr).split(' ')
    temp_list = [int(i) for i in temp_list]
    tag_list.append(temp_list)
    first_tag_list.append(temp_list[0])
    for thistag in temp_list:
        if (not(thistag in tag_counts)):
            tag_counts[thistag] = 0
        tag_counts[thistag] = tag_counts[thistag] + 1

questions_df['tags'] = tag_list
questions_df['first_tag'] = first_tag_list

tag_dict = {}
for thistag in tag_counts:
    temp_df = questions_df.loc[
        [(thistag in questions_df.tags[i]) for i in range(total_q)]
    ]
    temp_list = temp_df.content_id.tolist()
    tag_dict[thistag] = temp_list

#Delete all large variables not used past this part to conserve memory
del tag_list, first_tag_list

In [ ]:
#See the frequency of each tag
plt.figure(figsize=(12,6))
plt.plot(*zip(*sorted(tag_counts.items())))
plt.xlabel('Tag')
plt.ylabel('Count of questions with that tag')
plt.title('Tag Count')
plt.grid(True)

In [ ]:
#Get statistics per tag
stats_tag_dict = {}
for thistag in tag_dict:
    temp_df = train_df.loc[train_df.content_type_id==0]
    temp_df = temp_df.loc[temp_df['content_id'].isin(tag_dict[thistag])]
    tag_weight = len(temp_df)
    temp_df = temp_df.agg({'answered_correctly': [np.mean, np.std, 'skew']})
    if(not(thistag in stats_tag_dict)):
        stats_tag_dict[thistag] = {}
    stats_tag_dict[thistag]['tag_mean'] = temp_df.loc['mean'][0]
    stats_tag_dict[thistag]['tag_std'] = temp_df.loc['std'][0]
    stats_tag_dict[thistag]['tag_skew'] = temp_df.loc['skew'][0]
    stats_tag_dict[thistag]['weight'] = tag_weight

#For each question find the weighted average statistic
comb_tag_stat_list = []
for thislist in questions_df.tags:
    mean_list = []
    std_list = []
    skew_list = []
    weight_list = []
    for thistag in thislist:
        mean_list.append(stats_tag_dict[thistag]['tag_mean'])
        std_list.append(stats_tag_dict[thistag]['tag_std'])
        skew_list.append(stats_tag_dict[thistag]['tag_skew'])
        weight_list.append(stats_tag_dict[thistag]['weight'])
    if(weight_list[0]==0):
        comb_tag_stat_list.append(np.array([0.5, 0, 0]))
    else:
        temp_array = np.array([mean_list, std_list, skew_list])
        comb_tag_stat_list.append(np.average(temp_array, axis=1, weights=weight_list))

questions_df = questions_df.merge(
    pd.DataFrame(comb_tag_stat_list),
    left_index=True,
    right_index=True
)
questions_df.rename(
    columns={
        0: 'tag_mean',
        1: 'tag_std',
        2: 'tag_skew'
    },
    inplace=True
)
questions_df.drop(['tags'], axis=1, inplace=True)

time_finish = time.time()
print('Finished questions_df at', time.ctime())
print('Time elapsed:', int(time_finish - time_start), 'sec')

#Delete all large variables not used past this part to conserve memory
del stats_tag_dict, tag_dict, tag_counts

In [ ]:
#From the lectures metadata, create dictionary of part and tags
lectures_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
lectures_df.fillna(value=0)
lectures_df.drop(['type_of'], axis=1, inplace=True)
lectures_df.rename(columns={'lecture_id': 'content_id', 'tag': 'first_tag'}, inplace=True)
lectures_df['tag_mean'] = -1
lectures_df['tag_std'] = 0
lectures_df['tag_skew'] = 0

In [ ]:
#Define function to merge all dictionaries onto dataset
def merge_all_dict(in_df):
    temp_df = in_df.merge(content_id_dict, how='left', on='content_id')
    temp_df = temp_df.merge(container_dict, how='left', on='task_container_id')
    temp_df.loc[
        temp_df.content_type_id==1,
        ['container_mean', 'content_mean']
    ] = -1
    temp_df.loc[
        temp_df.content_type_id==1,
        ['container_std', 'container_skew', 'content_std', 'content_skew']
    ] = 0
    
    leconly_df = temp_df.loc[temp_df.content_type_id==1]
    leconly_df = leconly_df.merge(lectures_df, how='left', on='content_id')[
        ['row_id', 'first_tag', 'part', 'tag_mean', 'tag_std', 'tag_skew']
    ]
    queonly_df = temp_df.loc[temp_df.content_type_id==0]
    queonly_df = queonly_df.merge(questions_df, how='left', on='content_id')[
        ['row_id', 'first_tag', 'part', 'tag_mean', 'tag_std', 'tag_skew']
    ]
    
    temp_df = temp_df.merge(leconly_df, how='left', on='row_id').combine_first(queonly_df)
    
    return temp_df.fillna(value=0).reset_index(drop=True)

train_df = merge_all_dict(train_df)
print('After merge, (rows, columns) =', train_df.shape)

In [ ]:
#Quick glimpse at the data after processing
train_df.iloc[80:100]

In [ ]:
#Define the labels of the input vector
#Columns that need to be scaled are first
scaled_labels = [
    'timestamp',
    'prior_question_elapsed_time',
    'content_id',
    'part',
    'first_tag',
    'task_container_id'
]
stat_labels = [
    'content_mean',
    'content_std',
    'content_skew',
    'container_mean',
    'container_std',
    'container_skew',
    'tag_mean',
    'tag_std',
    'tag_skew'
]
other_labels = [
    'prior_question_had_explanation',
    'content_type_id',
    'answered_correctly'
]
all_labels = scaled_labels + stat_labels + other_labels
sindex = len(scaled_labels)

#Define input shapes
timesteps = 50
features = len(all_labels)

In [ ]:
#Create scaler for first set of labels
print('Creating scaler for preprocessing:')
scaler = StandardScaler()
scaler.fit(train_df[scaled_labels].to_numpy())

In [ ]:
#Define function that will get the relevant features for time series
#Will scale all features that need it
def build_ts_array(in_df):
    out_np = in_df[all_labels].to_numpy()
    out_np[:, 0:sindex] = scaler.transform(out_np[:, 0:sindex])
    return out_np

#Define function that will get features for current time sample
def build_ct_array(in_df):
    out_np = in_df[all_labels[:-2]].to_numpy()
    out_np[:, 0:sindex] = scaler.transform(out_np[:, 0:sindex])
    return out_np

In [ ]:
#Define function that returns samples of a given user
#Data will be front padded if the number of rows is less than timesteps
def user_sample_array(in_df, inuser, numsamples):
    temp_df = in_df.loc[in_df.user_id==inuser].reset_index(drop=True)
    ts_list = []
    ct_list = []
    y_list = []
    #.iloc includes the first argument, but excludes the second, thus ts is properly shifted
    if(len(temp_df)>1):
        for i in range(len(temp_df)-1, 0, -math.ceil(len(temp_df) / numsamples)):
            if(temp_df.iloc[i].content_type_id==1):
                continue
            next_ts = build_ts_array(temp_df.iloc[max(0, i-timesteps):i])
            next_ct = build_ct_array(temp_df.iloc[i:i+1])
            next_y = temp_df['answered_correctly'].iloc[i]
            if (len(next_ts) < timesteps):
                next_ts = np.vstack((
                    np.zeros((timesteps - len(next_ts), features)),
                    next_ts
                ))
            ts_list.append(next_ts)
            ct_list.append(next_ct)
            y_list.append(next_y)
    else:
        if(temp_df.content_type_id[0]==0):
            next_ts = np.zeros((timesteps, features))
            next_ct = build_ct_array(temp_df)
            next_y = temp_df['answered_correctly'].iloc[0]
            ts_list.append(next_ts)
            ct_list.append(next_ct)
            y_list.append(next_y)
    return ts_list, ct_list, y_list

In [ ]:
#Create the training set
xts_train_list = []
xct_train_list = []
y_train_list = []

#Building the set, each sample is the time series of one user
time_start = time.time()
print('Start creating training set at', time.ctime())
sampleperuser = 5
for thisuser in user_list:
    ts_list, ct_list, y_list = user_sample_array(train_df, thisuser, sampleperuser)
    for tssample, ctsample, ysample in zip(ts_list, ct_list, y_list):
        xts_train_list.append(tssample)
        xct_train_list.append(ctsample)
        y_train_list.append(ysample)

xts_train = np.array(xts_train_list)
xct_train = np.array(xct_train_list)
y_train = np.array(y_train_list)
xct_train = np.reshape(xct_train, (len(y_train), features-2))

time_finish = time.time()
print('Finished at', time.ctime())
print('Time elapsed:', int(time_finish - time_start), 'sec')
print('Shape of xts_train:', xts_train.shape)
print('Shape of xct_train:', xct_train.shape)
print('Shape of y_train:', y_train.shape)

#Delete all large variables not used past this point to conserve memory
del xts_train_list, xct_train_list, y_train_list

In [ ]:
#Show example time sample array for training
print('Time series example:')
for i, thislabel in enumerate(all_labels):
    print('{:>32}'.format(thislabel), end=': ')
    for val in xts_train[0, -5:, i]:
        print('{:>8.4f}'.format(val), end=' ')
    print('')
print('\nCurrent time example:')
for i, thislabel in enumerate(all_labels[:-2]):
    print('{:>32}'.format(thislabel), end=': ')
    print('{:>8.4f}'.format(xct_train[0, i]), end=' ')
    print('')
print('\nExpected result from model:', y_train[0])

In [ ]:
#Shuffle and split for validation
xts_train, xts_val, xct_train, xct_val, y_train, y_val = train_test_split(
    xts_train,
    xct_train,
    y_train,
    test_size = 1/10,
    random_state = 17,
    shuffle = True
)
print('After split:')
print('# of training data:', xts_train.shape[0])
print('# of validation data:', xts_val.shape[0])

In [ ]:
#Delete all large variables not used past this point to conserve memory
del train_df

# Model Creation and Training

In [ ]:
#Define hyperparameters
eta = 1E-2
totalepoch = 50

lstm_neurons = 50
ct_neurons = 10
dense_neurons = 10

lstm_r_reg = 1E-5
lstm_i_reg = 5E-4
ct_reg = 5E-4
comb_reg = 1E-5

In [ ]:
#Build model
from tensorflow.keras.optimizers import Adam

#Create inputs
ts_input = keras.Input(shape=(timesteps, features), name='ts_input')
ct_input = keras.Input(shape=(features-2), name='ct_input')

#Time series path
ts_mask = layers.Masking(
    mask_value=0.,
    name='ts_mask'
)(ts_input)
ts_state = layers.LSTM(
    lstm_neurons,
    recurrent_regularizer=regularizers.l1(lstm_r_reg),
    kernel_regularizer=regularizers.l1(lstm_i_reg),
    name='ts_lstm'
)(ts_mask)

#Current time path
ct_state = layers.Dense(
    ct_neurons,
    activation='relu',
    kernel_regularizer=regularizers.l1(ct_reg),
    name='ct_dense'
)(ct_input)

#Combined path
statemerge = layers.Concatenate(
    name='comb_concat'
)([ts_state, ct_state])
comb_dense = layers.Dense(
    dense_neurons,
    activation='relu',
    kernel_regularizer=regularizers.l1(comb_reg),
    name='comb_dense'
)(statemerge)
ans_pred = layers.Dense(
    1,
    activation='sigmoid',
    name='ans_pred'
)(comb_dense)

#Model compile
model = keras.Model(
    inputs=[ts_input, ct_input],
    outputs=[ans_pred],
    name='advancedmodel'
)

opt_adam = Adam(learning_rate=eta)
model.compile(
    optimizer=opt_adam,
    loss='mse',
    metrics=['AUC']
)
model.summary()

In [ ]:
#Fit model to training set
time_start = time.time()
print('Training start at', time.ctime())
history = model.fit(
    (xts_train, xct_train),
    y_train,
    batch_size=1024,
    epochs=totalepoch,
    validation_data=((xts_val, xct_val), y_val),
    verbose=0
)
time_finish = time.time()
print('Training finished at', time.ctime())
print('Time elapsed:', int(time_finish - time_start), 'sec')

In [ ]:
pd.DataFrame(history.history)[['auc','val_auc']].plot(figsize=(12,6))
plt.grid(True)
plt.gca().set_ylim(0.7, 0.82)
plt.title('Area Under Curve')
plt.show()

In [ ]:
pd.DataFrame(history.history)[['loss','val_loss']].plot(figsize=(12,6))
plt.grid(True)
plt.gca().set_ylim(0.18, 0.26)
plt.title('Loss/Error')
plt.show()

In [ ]:
#Apply model on the validation set, and show ROC curve
y_val_pred = model.predict((xts_val, xct_val)).ravel()
fpr_val, tpr_val, thresh_val = roc_curve(y_val.astype(np.uint8), y_val_pred)
auc_val = auc(fpr_val, tpr_val)
plt.figure(figsize=(10,10))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_val, tpr_val, label='Val (area = {:.5f})'.format(auc_val))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='center right')
plt.show()

In [ ]:
#Show example prediction
print('{:>10}'.format('Expected'), '{:>11}'.format('Prediction'))
for i in range(10):
    print('{:>10.7f}'.format(y_val[i]), '{:>11.7f}'.format(y_val_pred[i]))

In [ ]:
weights = model.trainable_weights
lstm_input_w = [0]*features
for i in range(features):
    temp_array = weights[0][i,::4]
    temp_array = [abs(j) for j in temp_array]
    lstm_input_w[i] = np.mean(temp_array)
lstm_input_w_scaled = 1000*(lstm_input_w/max(lstm_input_w))

print('Time series/LSTM feature relevance')
print('{:>30}'.format('FEATURE'), '{:>9}'.format('WEIGHT'), '{:>8}'.format('RELATIVE'))
for i, val in enumerate(lstm_input_w):
    print(
        '{:>30}'.format(all_labels[i]),
        '{:>9.6f}'.format(val),
        '{:>8}'.format(lstm_input_w_scaled[i].astype(int))
    )
    
dense_input_w = [0]*(features-2)
for i in range(features-2):
    temp_array = weights[3][i]
    temp_array = [abs(j) for j in temp_array]
    dense_input_w[i] = np.mean(temp_array)
dense_input_w_scaled = 1000*(dense_input_w/max(dense_input_w))

print('\nCurrent time feature relevance')
print('{:>30}'.format('FEATURE'), '{:>9}'.format('WEIGHT'), '{:>8}'.format('RELATIVE'))
for i, val in enumerate(dense_input_w):
    print(
        '{:>30}'.format(all_labels[i]),
        '{:>9.6f}'.format(val),
        '{:>8}'.format(dense_input_w_scaled[i].astype(int))
    )

In [ ]:
#Delete all large variables not used past this point to conserve memory
del xts_train, xct_train, y_train, xts_val, xct_val, y_val

# Model Predictions

In [ ]:
# The environment can only be created once, per Riiid rules
# If you need to rerun the script, first kill the session/kernel
env = riiideducation.make_env()

#Load test data. Each batch of tests is along the principle axis, and must be followed in a strict order (to emulate the passage of time).
#An iteration of test must call 'env.predict()' else there will be an error
iter_test = env.iter_test()

#Apply model, and place results in prediction_df with the 'env.predict()' function, such as below
#>>env.predict(prediction_df)
#After the iter_test tuple is exhausted, the environment will output the final prediction in submission.csv
test_user_dict = {}
old_user_list = []
old_row_list = []
old_test_df = pd.DataFrame({'empty': [0]})
for (test_df, prediction_df) in iter_test:
    
    #Preprocessing
    test_df = test_df.reset_index(drop=True)
    #Get answers from previous group
    pgac_list = eval(str(test_df['prior_group_answers_correct'].iloc[0]))
    #Drop columns
    test_df.drop(
        ['prior_group_responses', 'prior_group_answers_correct'],
        axis=1,
        inplace=True
    )
    df_preprocess(test_df)
    #Get user list and row list for this group
    new_user_list = test_df.user_id.tolist()
    new_row_list = test_df.row_id.tolist()
    #Merge dictionaries
    test_df = merge_all_dict(test_df)
    #Create answer column
    test_df['answered_correctly'] = [0.5]*len(test_df)
    test_df.fillna(value=0, inplace=True)
    
    #Update answered_correctly in previous group, then insert to dictionary
    if(pgac_list):
        pgac_dict = pd.DataFrame.from_dict(
            {'row_id': old_row_list, 'answered_correctly': pgac_list}
        )
        old_test_df = old_test_df.merge(pgac_dict, how='left', on='row_id')
        for thisindex, (thisrow, thisuser) in enumerate(zip(old_row_list, old_user_list)):
            if(not(thisuser in test_user_dict)):
                test_user_dict[thisuser] = {'rows': []}
            test_user_dict[thisuser][thisrow] = old_test_df.iloc[thisindex].tolist()
            test_user_dict[thisuser]['rows'].append(thisrow)
            if(len(test_user_dict[thisuser]['rows']) > timesteps):
                row_remove = test_user_dict[thisuser]['rows'].pop(0)
                test_user_dict[thisuser].pop(row_remove)    
    
    #Predict for each user in group
    xts_test_list = []
    xct_test_list  =[]
    prediction_rows = []
    for thisrow, thisuser in zip(new_row_list, new_user_list):
        temp_df = test_df.loc[test_df.row_id==thisrow]
        if(temp_df.content_type_id.iloc[0]==0):
            if(thisuser in test_user_dict):
                temp_dict = dict(test_user_dict[thisuser])
                temp_dict.pop('rows')
                database_df = pd.DataFrame.from_dict(
                    temp_dict,
                    orient='index',
                    columns=test_df.columns
                )
                user_df = pd.concat([database_df, temp_df])
            else:
                user_df = temp_df
            next_ts, next_ct, next_y = user_sample_array(user_df, thisuser, 1)
            xts_test_list.append(next_ts[0])
            xct_test_list.append(next_ct[0])
            prediction_rows.append(thisrow)
        
    xts_test = np.array(xts_test_list)
    xct_test = np.array(xct_test_list)
    xct_test = np.reshape(xct_test, (xts_test.shape[0], features-2))
    pred_list = model.predict((xts_test, xct_test)).ravel()
    update_df = pd.DataFrame({'row_id': prediction_rows, 'prediction': pred_list})
    update_df.fillna(0.5, inplace=True)

    #Save new -> old
    old_test_df = test_df.drop(['answered_correctly'], axis=1)
    old_user_list = list(new_user_list)
    old_row_list = list(new_row_list)
    
    #Call env.predict()
    prediction_df = prediction_df.merge(update_df, how='left', on='row_id')
    prediction_df.drop(['answered_correctly'], axis=1, inplace=True)
    prediction_df.rename(columns={'prediction': 'answered_correctly'}, inplace=True)
    prediction_df = prediction_df.astype({'answered_correctly': 'float64'})
    prediction_df.fillna(0.5, inplace=True)
    env.predict(prediction_df)